In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt

def generate_random_moves():
    """
    Generates random moves for each controller
    """
    individual_controller = []
    for x in range(num_of_moves):
        individual_controller.append(random.choice(types_of_moves))
    return individual_controller

def generate_controllers_population():
    controller_pop = []  
    """ Generates initial controllers population """
    for x in range(controllers_pop_size):  
        individual_controller = generate_random_moves()
        controller_pop.append(individual_controller)    
    return controller_pop

def play_game(individual):
    
    """
    Runs the game playing the move that determined
    by random solutions and gets the best tile and sum of all tiles
    """
    b = Game().b 
    j = 0
    best_tile = 0
    
    for j in range(len(individual)): 

        action = individual[j]

        if   action == 1 : 
            b = Game.left(b)
        elif action == 2 : 
            b = Game.right(b) 
        elif action == 3 : 
            b = Game.up(b) 
            b = Game.down(b)        
        elif action == 4 : 
            b = Game.down(b)        

        b_copy = b.copy()
        b = Game.add_random_tiles(b, 1)

        if b == None:
            b = b_copy

        if j == num_of_moves - 1:
            best_tile    = max(x for row in b for x in row)
            sum_of_tiles = sum(x for row in b for x in row)
            break 
            
    if print_output == 1:
        print(Game.print_board(b))
    return best_tile, sum_of_tiles

def tournament_selection():  
    """ Selects 2 random controllers from the controllers population, 
    compares if both are same, then try to find different controllers 
    until not found or termination condition reached, in this case
    while_loop_index < controllers_pop_size, to avoid while loop to go in infinite loop.
    Returns controller with better tile.
    """
    
    while_loop_index = 0
    rand_soln1 = random.choice(pop_with_fitness) # Select random solution 1
    rand_soln2 = random.choice(pop_with_fitness) # Select random solution 2

    while(rand_soln1 == rand_soln2 and while_loop_index < controllers_pop_size):
        rand_soln1 = random.choice(pop_with_fitness)            
        rand_soln2 = random.choice(pop_with_fitness)            
        while_loop_index += 1
        
    if rand_soln1[1] > rand_soln2[1]:            
        return rand_soln1
    else:
        return rand_soln2

def crossover(child_1, child_2):
        """ Performs crossover on the parents passed 
        as parameter to produce new offspring 
        This will be done based on cross_over parameter"""
        
        for i in range(cross_over):
            k = random.randint(0, len(child_1) - 1)                    
            
            for i in range(k, len(child_1)):
                child_1[i], child_2[i] = child_2[i], child_1[i]   ### Crossover ###
        
        return child_1, child_2

def mutation(child):
    
    """ Performs mutation on the child passed 
        as parameter to produce new offspring.
        This will be done based on the Mutation parameter"""
    
    while_loop_index = 0
    
    for i in range(mutation_rate):
        k = random.randint(0, len(child) - 1)           ### Get random gene from Child Chromosome ###
        j = random.randint(0, len(types_of_moves) - 1)  ### Get random number from types_of_moves to replace with child gene ###
        
        while (child[k] == types_of_moves[j] and while_loop_index < controllers_pop_size):            
            k = random.randint(0, len(child) - 1)  
            j = random.randint(0, len(types_of_moves) - 1)  

            while_loop_index += 1
        child[k] = types_of_moves[j] 
        
    return child

def weakest_replacement(child):
    
    """ Evaluates best tile of new child controller passed as a parameter 
    and then replace the weakest from the controllers population with it, if present """
    
    best_tile, sum_of_tiles = play_game(child)    
    
    weakest_index = 0
    weakest_found_ind = 0
    
    for i in range(len(pop_with_fitness)):
        if (pop_with_fitness[i][1] < best_tile):
            weakest_found_ind = 1
            
            if (pop_with_fitness[i][1] < pop_with_fitness[weakest_index][1]):
                weakest_index = i
   
    if weakest_found_ind == 1:
        pop_with_fitness[weakest_index] = [child,best_tile,sum_of_tiles]
    child = []

def calc_average():
    """ Calulates the average of sum of tiles of each controller and saves it in a global list, 
    which will be used in plotting the graph """
    
    sum_scores = 0
    
    for x in range(len(pop_with_fitness)):
        sum_scores += pop_with_fitness[x][2]
    avg = sum_scores/len(pop_with_fitness)

    avg_fitness.append(avg)    
    
def Evolve():
    
    """
    Performs evolution process by doing tournament selection,
    crossover, mutation and weakest replacement to find the
    best controllers.
    """
    
    for i in range(iterations):
        
        ### Tournament Selection - Starts ###
        parent_solns = []
        while_loop_index = 0
        parent_soln1 = tournament_selection()
        parent_soln2 = tournament_selection()

        while (parent_soln1 == parent_soln2 and while_loop_index < controllers_pop_size):
            parent_soln1 = tournament_selection()
            parent_soln2 = tournament_selection()
            while_loop_index += 1
        parent_solns.append(parent_soln1)  
        parent_solns.append(parent_soln2)  
        #### Tournament Selection - Ends ###    

        ### Crossover - Starts ###
        child_1 = parent_solns[0][0].copy()
        child_2 = parent_solns[1][0].copy()

        if cross_over != 0:
            child_1, child_2 = crossover(child_1, child_2) 
        ### Crossover - Ends ###

        ### Mutation - Starts ###
        if mutation_rate != 0:
            child_1 = mutation(child_1)
            child_2 = mutation(child_2)
        ### Mutation - Ends ###

        ### Weakest replacement - Starts ###
        weakest_replacement(child_1)
        weakest_replacement(child_2)

        child_1 = []
        child_2 = []
        ### Weakest replacement - Ends ###

        calc_average()

    list_avg_fitness.append(avg_fitness)
    final_tiles = []
    final_scores = []
    for x in range(len(pop_with_fitness)):
        final_tiles.append(pop_with_fitness[x][1])
        final_scores.append(pop_with_fitness[x][2])
    best_final_tiles.append(max(final_tiles))
    best_final_scores.append(max(final_scores))

    controllers.clear()
    for i in range(len(pop_with_fitness)):  
        if print_output == 1:
            print("Controller " + str(i+1) + " after evolution with Best tile = " +str(pop_with_fitness[i][1]) \
                  + ", Sum of Tiles = " + str(pop_with_fitness[i][2]) + "\n")         
            
        controllers.append(pop_with_fitness[i][0].copy())
    
import random
import itertools

class Game:
    def __init__(self):
        """ Initialize a 2048 game, 4*4 matrix of zeros
        Add two random tiles on the board, as that's how game starts           
        """
        b = [[0]*4 for i in range(4)]
        self.b = Game.add_random_tiles(b, 2)
    
    def print_board(b):
        """ print the board in the form of matrix """
        
        return '\n'.join([''.join(['{:8}'.format(element) for element in row]) for row in b])

    def add_random_tiles(b, k=1):
        """ Add random tiles to the board based on the k value.
            Probabaility of 2 is 90%; and of 4 is 10% """

        rows, cols = list(range(4)), list(range(4))
        random.shuffle(rows)
        random.shuffle(cols)

        board_copy  = [[x for x in row] for row in b]
        
        probability_list  = [2]*9 + [4]
        counter = 0

        for i,j in itertools.product(rows, rows):
           
            if board_copy[i][j] != 0: continue
            
            board_copy[i][j] = random.sample(probability_list, 1)[0]
            
            counter += 1
            if counter == k  : return board_copy        
        
    def left(b):
        """ Returns a left merged board
        """

        return Game.combine(b)

    def right(b):
        """ Returns a right combined/merged board
        """

        def reverse(x):
            return list(reversed(x))
        
        b_reverse = map(reverse, iter(b))
        return [reverse(x) for x in Game.combine(b_reverse)]

    def up(b):
        """ Returns an upward combined board, zip(*b_trans) is transpose
        """

        b_trans = Game.left(zip(*b))
        return [list(x) for x in zip(*b_trans)]

    def down(b):
        """ Returns an downward combined board, zip(*b_trans) is transpose
        """
        
        b_trans = Game.right(zip(*b))
        return [list(x) for x in zip(*b_trans)]

    def combine(b):
        """ Returns a combined/merged board """
        
        def combine_helper(row, a):
            """
            Helper for combining/merging the tiles. If we're finished with the list,
            nothing to do; return. and if we have more than one cell, combine results of first
            with right if they match; skip over the right and continue merging
            """
            
            if not row:
                return a
            x = row[0]
            if len(row) == 1:
                return combine_helper(row[1:], a + [x])
            return combine_helper(row[2:], a + [2*x]) if x == row[1] else combine_helper(row[1:], a + [x])

        ret = []
        for row in b:
            combined = combine_helper([x for x in row if x != 0], [])
            combined = combined + [0]*(len(row)-len(combined))
            ret.append(combined)
        return ret


#### Program starts here #####    

#intial board   = [[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]]
#type of moves  = ["left" = 1, "right" = 2, "up" = 3, "down" = 4]
#random nums probability  = [2,2,2,2,2,2,2,2,2,4]
seed = 0
mutation_rate = 100
cross_over    = 75
num_of_moves  = 500
iterations    = 1000
trials        = 5
print_output  = 0
controllers_pop_size = 10
types_of_moves = [1,2,3,4]
controllers = []
pop_with_fitness = []
list_avg_fitness = []
best_final_tiles = []
best_final_scores = []
controllers  = generate_controllers_population()
game_df = pd.DataFrame()

for k in range(trials): 
    pop_with_fitness.clear()
    avg_fitness = []

    for i in range(len(controllers)): 
        best_scores = []         
        seed += 5
        random.seed(seed)
        
        # Initially each controller will play 1 game and it's score will be considered for initial population    
        best_tile = 0
        sum_of_tiles = 0
        
        best_tile,sum_of_tiles = play_game(controllers[i])            
        pop_with_fitness.append([controllers[i], best_tile, sum_of_tiles])        
        
        if print_output == 1:
            print("Controller " + str(i+1) + " before evolution with Best tile = " +str(pop_with_fitness[i][1]) \
                  + ", Sum of Tiles = " + str(pop_with_fitness[i][2]) + "\n")         
    
    Evolve()
    mutation_rate += 25
    cross_over    += 15
    new_row = {'Trial' : k + 1,'Mutation rate': mutation_rate/num_of_moves,
               'Crossover rate': cross_over/num_of_moves, 'Best Tile': best_final_tiles[k], 'Best Score': best_final_scores[k]}
    game_df = game_df.append(new_row, ignore_index=True)
    
    
for x in range(len(list_avg_fitness)):    
    plt.plot(list_avg_fitness[x], label="Trial " + str(x+1) + " with Best tile " +str(best_final_tiles[x]))

display(game_df)
    
plt.title('Average Score vs Iteration')
plt.xlabel('Iteration')
plt.ylabel('Average Score')
plt.legend()
plt.show()   